In [1]:
import sqlite3
import pandas as pd
import numpy as np
con = sqlite3.connect("dump.db")
cur = con.cursor()
UserPath = pd.read_sql("select * from UserPath", con)[:10000]

DatabaseError: Execution failed on sql 'select * from UserPath': no such table: UserPath

In [10]:
#это дропнуть для нормальной бд
new_actions = ['Like', 'Dislike', 'Favourite']
all_actions = ['EventShow', 'SuccessPurchase'] + new_actions
np.random.seed(42)  
random_actions = np.random.choice(all_actions, size=len(UserPath))
UserPath['Act'] = random_actions
#это дропнуть для нормальной бд

In [11]:
interaction_matrix = pd.DataFrame(0, index=UserPath['UserID'].unique(), columns=UserPath['EventID'].unique())
for _, row in UserPath.iterrows():
    if row['Act'] == 'Dislike':
        interaction_matrix.loc[row['UserID'], row['EventID']] = -1
    elif row['Act'] == 'EventPreBuy':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 1
    elif row['Act'] == 'Favourite':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 2
    elif row['Act'] == 'EventPreBuy':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 3
    elif row['Act'] == 'BuyWithContact':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 4
    elif row['Act'] == 'Like':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 5
    elif row['Act'] == 'SuccessPurchase':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 6

interaction_matrix = interaction_matrix.stack().reset_index()
interaction_matrix.columns = ['user_id', 'event_id', 'action']


In [12]:
interaction_matrix = interaction_matrix.pivot_table(index='user_id', columns='event_id', values='action', fill_value=0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc_matrix = sc.fit_transform(interaction_matrix)
interaction_matrix = pd.DataFrame(sc_matrix, index=interaction_matrix.index, columns=interaction_matrix.columns)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
user_similarity = cosine_similarity(interaction_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

In [14]:
def get_recommendations(user_id, interaction_matrix, user_similarity_df, n_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index.tolist()
    similar_users.remove(user_id)
    recommendations = {}
    for similar_user in similar_users:
        user_events = interaction_matrix.loc[similar_user]
        
        for event_id in user_events[user_events > 0].index:
            if interaction_matrix.loc[user_id][event_id] == 0:
                if event_id not in recommendations:
                    recommendations[event_id] = user_events[event_id]
                else:
                    recommendations[event_id] += user_events[event_id]
    
    recommended_events = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n_recommendations]
    
    return recommended_events

In [15]:
user_id = '000a1dfe6f4f3018eb805e8115a0b5fc'
recommended_events = get_recommendations(user_id, interaction_matrix, user_similarity_df)

print(f'Рекомендуемые события для пользователя {user_id}:')
for event in recommended_events:
    print(f"Событие ID: {event[0]}, Сумма оценок: {event[1]}")

Рекомендуемые события для пользователя 000a1dfe6f4f3018eb805e8115a0b5fc:
Событие ID: b637fd8eb5f8706e14cfd3bd5d1d4a5b, Сумма оценок: 112.20640702641617
Событие ID: 24cb7d4c61faee29c95af44ef4437ef0, Сумма оценок: 78.52812659593164
Событие ID: 4c12c156013e4c3a4035e4591ed73ec5, Сумма оценок: 78.52812659593164
Событие ID: 8330f09177fd12af0aca71089532926a, Сумма оценок: 78.52812659593164
Событие ID: 8f895e70ac3a9bbf0cb706cff452b659, Сумма оценок: 78.52812659593164
